### Edgar Moises Hernandez-Gonzalez
#### 24-07-20-26-08-20
#### Clasificacion binaria de imagenes motoras en EEG con CNN-2D + LSTM
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Espectrogramas STFT (3D)
##### Entrada 135x31

In [1]:
from google.colab import drive

In [19]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import time
from scipy.signal import spectrogram
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from keras.models import load_model

In [4]:
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/MI-EEG-B9T.csv", header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/MI-EEG-B9E.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/etiquetas_test_9.csv", header=None)

In [5]:
print(x_train.shape)
print(x_test.shape)
print(y_test.shape)

(400, 3000)
(320, 3000)
(320, 1)


In [6]:
# esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test
n_classes = len(np.unique(y_test))

print("n_classes:", n_classes)

n_classes: 2


In [7]:
#unir 3 canales
def unir_espectrogramas_vertical(data):
  fs = 250
  datos = np.zeros((data.shape[0],135,31))
  temporal = np.zeros((135,31))
  for i in range(data.shape[0]): #n muestras
    for j in range(3): #3 canales
      sig = data.iloc[i, j*1000:(j+1)*1000]
      #sig = sig[124:624]
      #espectrograma de STFFT
      f, t, Sxx = spectrogram(sig, fs=250, window='hann', nperseg=250,
                              noverlap=225, nfft=500, scaling='spectrum')
      temporal[j*45:(j+1)*45, :] = Sxx[16:61, :]
    datos[i] = temporal
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [8]:
#llamar a unir_espectrogramas_vertical(data)
inicio = time.time()

x_train = unir_espectrogramas_vertical(x_train)
x_test = unir_espectrogramas_vertical(x_test)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
300
0
100
200
300
Tiempo: 2.0308542251586914


In [9]:
print(x_test.shape)

(320, 135, 31)


In [10]:
#convertir a float
x_test = x_test.astype('float32')

#escalar los valores en un rango de 0 a 1 (normalizar)
x_test /= np.ceil(np.max(x_train))

In [11]:
#convertir de 3D a 5D
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1], x_test.shape[2], 1))

print(x_test.shape)

(320, 1, 135, 31, 1)


In [12]:
model = load_model('/content/drive/My Drive/BCI-IV-2b/Modelos/STFT_RNN_09.hdf5')

In [13]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_21 (TimeDis (None, 1, 135, 31, 4)     40        
_________________________________________________________________
time_distributed_22 (TimeDis (None, 1, 67, 15, 4)      0         
_________________________________________________________________
time_distributed_23 (TimeDis (None, 1, 67, 15, 4)      148       
_________________________________________________________________
time_distributed_24 (TimeDis (None, 1, 33, 7, 4)       0         
_________________________________________________________________
time_distributed_25 (TimeDis (None, 1, 924)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 4)                 14864     
_________________________________________________________________
dense_9 (Dense)              (None, 32)               

In [15]:
inicio = time.time()

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.3462378978729248


In [ ]:
print("loss: %.4f" % test_loss)
print("accuracy: %.4f" % test_acc)

loss: 0.4222
accuracy: 0.8344


In [ ]:
probabilidades = model.predict(x_test)
y_pred = np.argmax(probabilidades, 1) #1=fila

In [ ]:
kappa = cohen_kappa_score(y_test, y_pred)
print("kappa: %.4f " % kappa)

kappa: 0.6687 


In [ ]:
matriz_confusion = confusion_matrix(y_test, y_pred)
print("confusion matrix:\n", matriz_confusion)

confusion matrix:
 [[138  22]
 [ 31 129]]


In [18]:
inicio = time.time()

probabilidades = model.predict(x_test[0].reshape(1,1,135,31,1))
y_pred = np.argmax(probabilidades, 1) #1=fila

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.04192519187927246
